In [ ]:
import os,sys,string, time
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
from importlib import reload

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Variables_bins as Variables_bins
import Utilities.Functions as Functions
import Utilities.Sys_unc_variables as Sys_unc
from Utilities.ipython_exit import exit

print ('Success')


# Reading in .pkl files

In [ ]:
Params = {"Run":"run3", #The run number, so far either "run1" or "run3"
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":"overlay",
          "Load_standard_bkgs":False,
          "Load_DetVars":False, #overlay DetVar samples
          "Only_keep_common_DetVar_evs":True, #Removes events not in all DetVar samples
          "Load_Signal_DetVars":False, #ee DetVar samples
          "Load_data":False,
          "FLATTEN":True, #Have one row per reconstructed object in the analysis dataframe
          "only_presel":False, #Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "Load_truth_vars":False, 
          "Load_lepton_signal":False,
          "Load_lepton_dirac":False,
          'Load_pi0_signal_DetVars':True, #pi0 DetVar samples
          "Load_pi0_signal":False,
          "Load_pi0_dirac":False}

# Params, samples = Functions.create_sample_list(Params)
Params, samples = Functions.new_create_sample_list(Params)
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"

In [ ]:
samples_dict = Functions.Load_initial_pkls(samples, Params, loc_pkls, "_full_Finished")

for sample in samples_dict:
    print(f"{sample} length is " + str(len(samples_dict[sample])))
print("Loading these variables:")
print(samples_dict[sample].keys())

## Applying Software Trigger

In [ ]:
def Apply_swtrig(df):
    query="swtrig_pre>0"
    df.query(query)

def Apply_flash_shift(df, sample_type):
    #Only required for beam off, overlay, dirtoverlay and signal (i.e all except beamon data)
    if(sample_type=="beamoff"): df["flash_time"]=df["flash_time"] -0.359
    else: df["flash_time"]=df["flash_time"] - 0.304
    
Pre_swtrig_dict = {}
swtrig = {}

apply_swtrig = True
if(apply_swtrig): print("Applying software trigger!")
for sample in samples_dict:
    Pre_swtrig_dict[sample] = len(samples_dict[sample]) #Should weight this properly, i.e take sum of weights for the total number.
    if(apply_swtrig):
        # Apply_swtrig(samples_dict[sample])
        if Params["Run"]=="run1":
            swtrig[sample] = samples_dict[sample].query("swtrig_pre>0").copy()
        if Params["Run"]=="run3":
            swtrig[sample] = samples_dict[sample].query("swtrig_post>0").copy() #Took this from Owen's code, should check exactly what it's doing
    else: swtrig[sample] = samples_dict[sample].copy()
    if sample != "beamgood":
        Apply_flash_shift(swtrig[sample], sample)
        
for sample in samples_dict:
    swtrig_effic = 100*(len(swtrig[sample])/Pre_swtrig_dict[sample])
    print(f"{sample} efficiency is " + str(swtrig_effic) + "%")
    
print(swtrig.keys())

# Plotting variables before preselection

In [ ]:
#Want to have arrows drawn on for where the preselection cut goes
HNL_mass = "100_ee"
if Params["Load_pi0_signal"]:
    HNL_mass = f"{HNL_mass}_pi0" #only plotting one at a time
variable = 'topological_score'

samples_plotting = {"overlay":swtrig["overlay"],
                    "dirtoverlay":swtrig["dirtoverlay"],
                    "beamoff":swtrig["beamoff"],
                    "signal":swtrig[HNL_mass]}

if Params["Run"] == "run1":
    sample_norms = {"overlay":Constants.run1_POT_scaling_dict["overlay"], #WAS using SF_overlay_run1
                    "dirtoverlay":Constants.run1_POT_scaling_dict["dirtoverlay"],
                    "beamoff":Constants.run1_POT_scaling_dict["beamoff"],
                    "signal":Constants.run1_POT_scaling_dict[HNL_mass]}
    flat_sys_vars_dict = Sys_unc.run1_Sys_unc
elif Params["Run"] == "run3":
    sample_norms = {"overlay":Constants.run3_POT_scaling_dict["overlay"],
                    "dirtoverlay":Constants.run3_POT_scaling_dict["dirtoverlay"],
                    "beamoff":Constants.run3_POT_scaling_dict["beamoff"],
                    "signal":Constants.run3_POT_scaling_dict[HNL_mass]}
    flat_sys_vars_dict = Sys_unc.run3_Sys_unc

if Params["Load_data"] == True:
    data_dict = {"beamgood":samples_dict["beamgood"]}
    data_norm = {"beamgood":1.0} #No scaling as this is data, should have scaled MC to this POT.
    samples_plotting.update(data_dict)
    sample_norms.update(data_norm)
    
PT.HNL_scaling_calculator(samples_plotting,sample_norms)


In [ ]:
for sample in samples_plotting:
    print(sample)
    
print(Constants.Preselection_dict_for_plot)

## Plot data and MC

In [ ]:
samples_plotting["overlay"].keys()

In [ ]:
#Calculate sys
Calculate_sys = False
variable = 'topological_score'

reload(PT)
reload(Functions)
reload(Constants)
reload(Sys_unc)
reload(Variables_bins)

if Params["Run"] == "run1": 
    flat_sys_vars_dict = Sys_unc.run1_Sys_unc
    PLOTSCALE = 240000000
if Params["Run"] == "run3": 
    flat_sys_vars_dict = Sys_unc.run3_Sys_unc
    PLOTSCALE = 240000000
    
if isinstance(Constants.variable_names_dict[variable],str):x_string = Constants.variable_names_dict[variable]
else: x_string=variable
if variable=='trk_score_v':ylabel = 'Reconstructed objects'
else: ylabel='Events'
if variable=='nslice':xticks=[0,1]
else: xticks=[]

if (variable.startswith('min')) or (variable.startswith('max')):
    dropdupes=True
    print("Only keeping one entry per event for this plot!")
else: dropdupes=False

BINS = Variables_bins.Presel_bins[variable]
XLIMS = [BINS[0],BINS[-1]]

Log_var = Variables_bins.Log_vars[variable]
cut_line = 0.0
# arrow_place = Variables_bins.arrow_places[variable]
arrow_place = []
print("Arrow place is " + str(arrow_place))

Name = ""
if Params["Load_pi0_signal"] == True: Name = "pi0/"
if Calculate_sys==True: Sys_name = "with_sys"
if Calculate_sys==False: Sys_name = "no_sys"
if arrow_place != []: end_name = "_arrow"
else: end_name = ""

In [ ]:
savefig=False

print(f"Plotting {variable} for " + Params["Run"])

bins_test = [0,25.0]
bins_test = BINS
print("nbins " + str(len(bins_test)-1))

PT.Plot_preselection_variable_data(variable, samples_plotting, sample_norms, xlabel=x_string, xlims=XLIMS, bins=bins_test, HNL_mass=100,
                                   HNLplotscale=PLOTSCALE,cutline=cut_line,legloc="upper right", show_ev_nums=False, CalcSys=Calculate_sys,logy=Log_var, 
                                   xticks = xticks, sys_dict=flat_sys_vars_dict, centre_bins=False, hatch=False, ylabel=ylabel, 
                                   arrow_place=arrow_place, legsize=16, dropdupes=dropdupes, err_print=False, chi_squared=False, 
                                   dirt_frac_error=0.9)

if savefig == True:
    plt.savefig(f"plots/Data_MC_comparison/{Name}Presel_{variable}_" + Params["Run"] + f"_{HNL_mass}MeV_{Sys_name}{end_name}.png")
    plt.savefig(f"plots/Data_MC_comparison/{Name}Presel_{variable}_" + Params["Run"] + f"_{HNL_mass}MeV_{Sys_name}{end_name}.pdf")

## Plotting only highest E object

In [ ]:
Highest_E_placeholder = Functions.only_keep_highest_E(samples_dict[HNL_mass])

samples_plotting = {"overlay":samples_dict["overlay"],
           "dirtoverlay":samples_dict["dirtoverlay"],
           "beamoff":samples_dict["beamoff"],
           "signal":Highest_E_placeholder}

PT.Plot_preselection_variable(variable, samples_plotting, sample_norms, xlabel=[],xlims=[1,100],bins=100, HNL_mass=HNL_mass,
                              HNLplotscale=600000,cutline=0.0,legloc="upper right")

plt.savefig("plots/Preselection_variables/" + Params["Run"] + "_" + str(HNL_mass) + "_MeV_" + str(variable)+"_highest_E.png")

## Plot only signal samples before preselection

In [ ]:
#Need a new plotting function for this
reload(Constants)

# Plotting significance of individual cuts

In [ ]:
# cut_start_str = "flash_time > 16.5"
reload(Functions)
cut_start_str = "topological_score < "

variable = "topological_score"

Sig_dict_max, Sig_dict_min, var_sig_max, var_sig_min = Functions.Significance_scan(Params, swtrig, cut_start_str, 0.5, 1.0, numsteps=20)

PT.Plot_significance_scan(variable, Sig_dict_max, Sig_dict_min, var_sig_max, var_sig_min)

# Pre-selection cuts

In [ ]:
Use_tight_presel=False

if Params["FLATTEN"] == True: Preselection_dict = Constants.Preselection_dict 
else: Preselection_dict = Constants.Preselection_dict_for_plot

if Use_tight_presel == True: Preselection_dict = Constants.Tight_Preselection_dict
        
if Params["Run"] == "run3": Preselection_dict.update(Constants.Preselection_dict_crtveto) #Adding crtveto cut
        
if (Params["Load_DetVars"] == True) and (Params["Run"] == "run3"):
    removed=Preselection_dict.pop('Fiducial_cut')

    
# Preselection_dict.update({"topological_score":"topological_score < 0.98"})

print("Preselection is:" + "\n")
print(Preselection_dict)


In [ ]:
reload(Functions)
sample_dict_copy = {}
for sample in swtrig:
    sample_dict_copy[sample] = swtrig[sample].copy()

    
if (Params["Load_DetVars"] == True) or (Params["Load_Signal_DetVars"] == True) or (Params['Load_pi0_signal_DetVars'] == True):
    Preselected = Functions.Preselection_DetVars(sample_dict_copy, Preselection_dict)
else:
    if Params["FLATTEN"] == False:
        Efficiency_dict, Preselected = Functions.Preselection_weighted_efficiency(sample_dict_copy, Preselection_dict)
    if Params["FLATTEN"] == True:
        Efficiency_dict, Preselected = Functions.Flattened_Preselection_weighted_efficiency(sample_dict_copy, Preselection_dict, Params["Run"])
    
    print(Efficiency_dict.keys())
print("Finished pre-selection cuts.")


## Printing and plotting efficiencies

In [ ]:
Preselection_signal_min, Preselection_signal_max = Functions.Get_signal_efficiency_range(Params, Preselection_dict, Efficiency_dict)

Functions.Print_efficiency_numbers(Params, Preselected, Efficiency_dict)


In [ ]:
reload(Constants)

In [ ]:
PT.Plot_preselection_efficiency(Params, Preselection_dict, Efficiency_dict, Preselection_signal_min, Preselection_signal_max, log=True)


In [ ]:
effic_wrt_prev, lowest_signal_wrt_prev, highest_signal_wrt_prev = Functions.Get_effic_wrt_previous(Params, Preselection_dict, Efficiency_dict)

PT.Plot_effic_wrt_previous(Params, Preselection_dict, effic_wrt_prev, lowest_signal_wrt_prev, highest_signal_wrt_prev)


## Only select highest energy object and "Prepare" for xgboost

In [ ]:
trk_score_cut = True

Prepared_dict = {}
if(trk_score_cut): Preselected_dict = Functions.Remove_high_trk_score_objects(Preselected, threshold=0.97)
else: Preselected_dict = Preselected.copy()

Highest_E_placeholder = Functions.only_keep_highest_E(Preselected_dict)
# Prepared_dict = Functions.Prepare_dfs_for_xgb(Highest_E_placeholder)
Prepared_dict = Functions.Fixed_Prepare_dfs_for_xgb(Highest_E_placeholder)

print(Prepared_dict.keys())

In [ ]:
# for variable in Prepared_dict["overlay"].keys():
#     print(variable)
#     print(max(Prepared_dict["overlay"][variable]))
#     print(min(Prepared_dict["overlay"][variable]))

# Save pkls

In [ ]:
print_vals = input("Do you want to save the pickle files? y/n ")
if print_vals == "y":
    Functions.Save_preselected_pkls(Prepared_dict, Params, loc_pkls, "full_Finished")
else:
    print("Not saving pickle files, move onto next cell")

## Plotting single mass point and background

In [ ]:
#Plotting efficiency
signal_mass = 220
plt.figure(figsize=[10,10])
plt.plot(np.array(range(1, len(Efficiency_dict[effic])+1)),Efficiency_dict[signal_mass],label=f"{signal_mass} MeV HNL",lw=4,markersize=15)
plt.plot(np.array(range(1, len(Efficiency_dict[effic])+1)),Efficiency_dict['overlay'],label="MC run1 overlay",lw=4,markersize=15)
plt.plot(np.array(range(1, len(Efficiency_dict[effic])+1)),Efficiency_dict['dirtoverlay'],label="MC run1 dirt",lw=4,markersize=15)
plt.plot(np.array(range(1, len(Efficiency_dict[effic])+1)),Efficiency_dict['beamoff'],label="MC run1 EXT",lw=4,markersize=15)
plt.ylabel("Fraction Selected")
plt.xticks(np.array(range(1, len(Efficiency_dict[effic])+1)),["Full sample"]+list(Preselection_dict.keys()),rotation=80)
#plt.yscale("log")
plt.legend()


## Plotting just signal

In [ ]:
plt.figure(figsize=[10,10])

for HNL_mass in Constants.HNL_mass_samples:
    plt.plot(np.array(range(1, len(Efficiency_dict[effic])+1)),Efficiency_dict[HNL_mass],label=f"{HNL_mass} MeV HNL",lw=4,markersize=15)
    
plt.ylabel("Fraction Selected")
plt.xticks(np.array(range(1, len(Efficiency_dict[effic])+1)),["Full sample"]+list(Preselection_dict.keys()),rotation=80)
# plt.yscale("log")
plt.legend()

plt.savefig("plots/Preselection_efficiencies/Signal_only_"+Params["Run"]+"_"+Params["Flat_state"]+".png")

# Testing code

## Making sum of weights (only for unflattened)

In [ ]:
def Sum_weights(df):
    weight = df["weight"] #Just one value per event
    sum_weights = sum(weight)
    return sum_weights

for sample in ['overlay', 'dirtoverlay']:
    sum_weights_placeholder = Sum_weights(samples_dict[sample])
    print(f"{sample} sum weights is " + str(sum_weights_placeholder))

In [ ]:
#Testing what the variables look like
variable = "dvtx_x_boundary"
def test_variables(df):
    test_signal = df[variable]
    print(type(test_signal))
    print(len(test_signal))
test_variables(df_overlay_run1)
test_variables(df_dirt_run1)
test_variables(df_EXT_run1)
test_variables(df_signal_run1)

In [ ]:
#print(overlay_from_pkl['nslice'])
#print("Flat Overlay .pkl is "+str(len(FLAT_overlay_from_pkl))+" entries long.")

print("-----Unflattened-----")
unique_var = overlay_from_pkl["nslice"].unique()
print(unique_var)
unique_var = overlay_from_pkl["n_pfps"].unique()
print(unique_var)
print(overlay_from_pkl["n_pfps"])
print(type(overlay_from_pkl["n_pfps"][0]))

# for var in overlay_from_pkl.keys():
#     if type(overlay_from_pkl[var][1]) == "numpy.ndarray":
#         print(var)

# print("-----Flattened-----")
# unique_var = FLAT_overlay_from_pkl["nslice"].unique()
# print(unique_var)
# unique_var = FLAT_overlay_from_pkl["n_pfps"].unique()
# print(unique_var)
# print(FLAT_overlay_from_pkl["n_pfps"])
# print(type(FLAT_overlay_from_pkl["n_pfps"]))

## Depracated code